In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import librosa
# import tensorflow as tf
import glob
import numpy as np

In [3]:
# follow hyperparameters from here, https://github.com/pytorch/fairseq/tree/master/examples/wav2vec

features = [(512, 10, 5), (512, 8, 4), (512, 4, 2), (512, 4, 2), (512, 4, 2), (512, 1, 1), (512, 1, 1)]
aggs = [(512, 2, 1), (512, 3, 1), (512, 4, 1), (512, 5, 1), (512, 6, 1), (512, 7, 1), (512, 8, 1), (512, 9, 1), 
        (512, 10, 1), (512, 11, 1), (512, 12, 1), (512, 13, 1)]
num_negatives = 10
prediction_steps = 12
learning_rate = 1e-5
min_learning_rate = 1e-9
max_learning_rate = 0.005
learning_scheduler = 'cosine'
max_update = 400000
residual_scale = 0.5
log_compression = True
warmup_updates = 50
warmup_init_lr = 1e-07
batch_size = 32
epoch = 10
total_steps = batch_size * epoch

In [4]:
import torch
from torch import nn
import torch.functional as F

In [5]:
np.random.seed(1)
x = np.random.normal(size = (2, 10, 5))
x

array([[[ 1.62434536, -0.61175641, -0.52817175, -1.07296862,
          0.86540763],
        [-2.3015387 ,  1.74481176, -0.7612069 ,  0.3190391 ,
         -0.24937038],
        [ 1.46210794, -2.06014071, -0.3224172 , -0.38405435,
          1.13376944],
        [-1.09989127, -0.17242821, -0.87785842,  0.04221375,
          0.58281521],
        [-1.10061918,  1.14472371,  0.90159072,  0.50249434,
          0.90085595],
        [-0.68372786, -0.12289023, -0.93576943, -0.26788808,
          0.53035547],
        [-0.69166075, -0.39675353, -0.6871727 , -0.84520564,
         -0.67124613],
        [-0.0126646 , -1.11731035,  0.2344157 ,  1.65980218,
          0.74204416],
        [-0.19183555, -0.88762896, -0.74715829,  1.6924546 ,
          0.05080775],
        [-0.63699565,  0.19091548,  2.10025514,  0.12015895,
          0.61720311]],

       [[ 0.30017032, -0.35224985, -1.1425182 , -0.34934272,
         -0.20889423],
        [ 0.58662319,  0.83898341,  0.93110208,  0.28558733,
          0.8

In [6]:
x = torch.from_numpy(x)
x.shape

torch.Size([2, 10, 5])

In [7]:
bsz, fsz, tsz = x.shape
bsz, fsz, tsz

(2, 10, 5)

In [8]:
y = x.transpose(0, 1)
y = y.contiguous().view(fsz, -1)
y.shape

torch.Size([10, 10])

In [9]:
high = tsz
n_negatives = 10
# neg_idxs = torch.randint(low=0, high=high, size=(bsz, n_negatives * tsz))
# neg_idxs

In [10]:
neg_idxs = torch.from_numpy(np.array([[2, 3, 2, 1, 4, 4, 4, 2, 3, 4, 2, 3, 1, 3, 3, 1, 2, 0, 2, 4, 2, 2, 2, 2,
         2, 4, 0, 0, 3, 3, 4, 0, 4, 4, 4, 2, 4, 4, 3, 2, 2, 4, 0, 0, 2, 4, 4, 4,
         2, 0],
        [1, 2, 4, 2, 2, 0, 0, 0, 3, 3, 0, 3, 1, 3, 4, 4, 3, 1, 4, 4, 0, 1, 3, 0,
         4, 0, 0, 0, 3, 3, 2, 4, 1, 0, 1, 2, 0, 2, 0, 0, 2, 2, 3, 1, 3, 4, 3, 4,
         1, 0]]))

In [11]:
for i in range(1, bsz):
    neg_idxs[i] += i * high
    
neg_idxs

tensor([[2, 3, 2, 1, 4, 4, 4, 2, 3, 4, 2, 3, 1, 3, 3, 1, 2, 0, 2, 4, 2, 2, 2, 2,
         2, 4, 0, 0, 3, 3, 4, 0, 4, 4, 4, 2, 4, 4, 3, 2, 2, 4, 0, 0, 2, 4, 4, 4,
         2, 0],
        [6, 7, 9, 7, 7, 5, 5, 5, 8, 8, 5, 8, 6, 8, 9, 9, 8, 6, 9, 9, 5, 6, 8, 5,
         9, 5, 5, 5, 8, 8, 7, 9, 6, 5, 6, 7, 5, 7, 5, 5, 7, 7, 8, 6, 8, 9, 8, 9,
         6, 5]])

In [12]:
negs = y[..., neg_idxs.view(-1)]
negs.shape

torch.Size([10, 100])

In [13]:
negs = negs.view(fsz, bsz, n_negatives, tsz).permute(2, 1, 0, 3)
negs.shape

torch.Size([10, 2, 10, 5])

In [14]:
negs[0]

tensor([[[-0.5282, -1.0730, -0.5282, -0.6118,  0.8654],
         [-0.7612,  0.3190, -0.7612,  1.7448, -0.2494],
         [-0.3224, -0.3841, -0.3224, -2.0601,  1.1338],
         [-0.8779,  0.0422, -0.8779, -0.1724,  0.5828],
         [ 0.9016,  0.5025,  0.9016,  1.1447,  0.9009],
         [-0.9358, -0.2679, -0.9358, -0.1229,  0.5304],
         [-0.6872, -0.8452, -0.6872, -0.3968, -0.6712],
         [ 0.2344,  1.6598,  0.2344, -1.1173,  0.7420],
         [-0.7472,  1.6925, -0.7472, -0.8876,  0.0508],
         [ 2.1003,  0.1202,  2.1003,  0.1909,  0.6172]],

        [[-0.3522, -1.1425, -0.2089, -1.1425, -1.1425],
         [ 0.8390,  0.9311,  0.8851,  0.9311,  0.9311],
         [ 1.2529,  0.5129,  0.4885,  0.5129,  0.5129],
         [ 1.1316,  1.5198, -1.3965,  1.5198,  1.5198],
         [-0.5045,  0.1600,  0.3156,  0.1600,  0.1600],
         [-0.3062,  0.8280,  0.7620,  0.8280,  0.8280],
         [-0.2008,  0.1866,  0.1983,  0.1866,  0.1866],
         [-0.6707,  0.3776,  1.1295,  0.3776, 

In [15]:
y = x[:].unsqueeze(0)
print(y.shape, negs.shape)
targets = torch.cat([y, negs], dim=0)
targets.shape

torch.Size([1, 2, 10, 5]) torch.Size([10, 2, 10, 5])


torch.Size([11, 2, 10, 5])

In [27]:
s

tensor([[[[[-0.0199,  0.1676,  0.2382,  ..., -0.1588,  0.2736,  0.0313],
           [-0.1879, -0.1027, -0.2801,  ...,  0.4280,  0.0158, -0.0910],
           [-0.3103,  0.1950, -0.2168,  ...,  0.3265,  0.0191, -0.1527],
           [ 0.0746, -0.0410,  0.0871,  ..., -0.0122,  0.0848,  0.1434],
           [ 0.0708,  0.0985,  0.0641,  ..., -0.0735,  0.1588, -0.0396]],

          [[ 0.0132, -0.0440, -0.0432,  ...,  0.1687, -0.0102,  0.1108],
           [ 0.0868,  0.0774,  0.0406,  ...,  0.0238, -0.0023, -0.0307],
           [ 0.0830,  0.0320,  0.2202,  ..., -0.0102,  0.0687, -0.0692],
           [-0.0151,  0.3591,  0.2524,  ..., -0.2356,  0.1166,  0.0866],
           [ 0.0677, -0.0997, -0.0691,  ...,  0.1018,  0.0365, -0.1243]],

          [[ 0.1786, -0.0690,  0.3055,  ..., -0.1702,  0.3010, -0.1287],
           [-0.3102,  0.1631, -0.2399,  ...,  0.2156,  0.0005,  0.1792],
           [-0.1526, -0.1865, -0.2420,  ...,  0.2012,  0.1100, -0.0034],
           [ 0.1693, -0.1523, -0.0617,  ...,  0

In [16]:
project_to_steps = nn.ConvTranspose2d(10, 10, (1, 12))
print(x.shape)
s = project_to_steps(x.unsqueeze(-1).float()).unsqueeze(0).expand(targets.size(0), -1, -1, -1, -1)
s.shape

torch.Size([2, 10, 5])


torch.Size([11, 2, 10, 5, 12])

In [17]:
import math

jin = 0
rin = 0
for _, k, stride in features:
    if rin == 0:
        rin = k
    rin = rin + (k - 1) * jin
    if jin == 0:
        jin = stride
    else:
        jin *= stride
offset = math.ceil(rin / jin)

offset = int(offset)
print(offset)

3


In [18]:
copies, bsz, dim, tsz, steps = s.shape
steps = min(steps, tsz - offset)
predictions = s.new(bsz * copies * (tsz - offset + 1) * steps - ((steps + 1) * steps // 2) * copies * bsz)
labels = torch.zeros_like(predictions)
predictions.shape, labels.shape

(torch.Size([66]), torch.Size([66]))

In [19]:
s.shape, targets.shape, s[..., :-offset, i].shape, targets[..., offset:].shape

(torch.Size([11, 2, 10, 5, 12]),
 torch.Size([11, 2, 10, 5]),
 torch.Size([11, 2, 10, 2]),
 torch.Size([11, 2, 10, 2]))

In [25]:
s[0].shape

torch.Size([2, 10, 5, 12])

In [20]:
start = end = 0
for i in range(steps):
    offset = i + offset
    end = start + (tsz - offset) * bsz * copies
    pos_num = (end - start) // copies
    print(start, pos_num, end, offset, s[..., :-offset, i].shape, targets[..., offset:].shape)
    predictions[start:end] = (s[..., :-offset, i].float() * targets[..., offset:].float()).sum(dim=2).flatten()
    labels[start:start + pos_num] = 1.
    start = end
    
labels

0 4 44 3 torch.Size([11, 2, 10, 2]) torch.Size([11, 2, 10, 2])
44 2 66 4 torch.Size([11, 2, 10, 1]) torch.Size([11, 2, 10, 1])


tensor([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
predictions

tensor([ 5.9004e-01, -6.5989e-01,  5.3224e-01, -1.0226e-01, -3.7656e-01,
        -6.5989e-01,  4.2338e-01, -1.6779e-02,  5.9004e-01, -6.5989e-01,
         5.3224e-01,  2.2175e-01,  5.9004e-01,  1.8709e-01,  5.3224e-01,
        -1.0226e-01,  9.0795e-01, -6.5989e-01, -3.8466e-01, -1.0226e-01,
         9.0795e-01, -6.4663e-01, -2.4600e-01, -1.0226e-01,  5.9004e-01,
         1.8709e-01,  5.3224e-01,  2.2175e-01,  5.9585e-01, -6.5989e-01,
        -2.4600e-01,  2.5392e-01,  5.9004e-01, -6.4663e-01, -2.4600e-01,
         1.0426e-01,  3.2566e-01, -6.4663e-01,  6.6557e-01,  2.2175e-01,
         9.0795e-01, -1.2358e+00,  6.6557e-01,  1.0426e-01,  7.8751e-01,
        -3.6611e-02,  7.8751e-01,  2.7267e-01,  7.8751e-01,  4.0492e-01,
         7.5874e-01, -3.6611e-02,  7.8751e-01, -3.6611e-02,  7.2576e-01,
        -3.6611e-02,  7.5874e-01,  4.0492e-01,  7.8751e-01,  5.3117e-04,
         7.2576e-01, -2.7355e-01,  7.2576e-01,  4.0492e-01,  1.6555e-01,
        -2.7355e-01], grad_fn=<CopySlices>)

In [22]:
torch.sign(predictions)

tensor([ 1., -1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,
         1., -1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1.,
         1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,
         1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,
         1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1.],
       grad_fn=<SignBackward>)